# Automation
**Prompting Guidelines**
- sampling 75% of personas
- 2 calls per persona
  
__ChatGPT 3.5 Turbo__
- input: \$0.50/1M tokens
- output: \$1.50/1M tokens

__Claude Sonnet/Haiku__
- input: \$0.25/1M tokens
- output: \$1.25/1M tokens

In [2]:
import pandas as pd

df = pd.read_csv('demographic_personas.csv')
sampled_df = df.sample(frac=0.75, random_state=42)  # Set random_state for reproducibility
print("Number of rows in sampled DataFrame:", sampled_df.shape[0])

Number of rows in sampled DataFrame: 688


In [3]:
gsam_prompt = """
The person comes across a stranger who is sitting slumped in a doorway head down, eyes closed, not moving. 
The person hears the stranger coughing and groaning.
"""
q = """
Please provide a value from 0-100 on the person’s inclination to help the stranger. 
0 indicates the person is not at all likely to help to stranger, while 100 indicates the person is very likely to help the stranger.

If any, provide a value for how many minutes the person stays and helps the stranger. 

If any, write a brief story about what actions the person takes.

Please format your answer like so:
Helpfulness: x (integer between 0-100)
Minutes waited: y (integer value)
Actions:
- Person does/does not z.
"""

## Claude

In [ ]:
from anthropic import Anthropic
import os
import json
import time
import pandas as pd

claude_df = pd.read_csv('claude_responses.csv')
print(claude_df)

delay = 15

def makeApiCall(prompt, num):
    auth_pers = json.loads(open("api_auth.txt").read()) # add api auth
    os.environ["ANTHROPIC_API_KEY"] = auth_pers['anthropic']['secretKey']
    client = Anthropic()
    
    full_context = prompt + gsam_prompt + q + num
    message = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=300,
        temperature=0.0,
        system="You are a helpful observer who can provide insight on people.",
        messages=[
            {"role": "user", "content": full_context}
        ]
    )
    return message.content[0].text


for index, row in claude_df.iterrows():
    prompt = row['Prompt'] # get prompt
    # print(prompt)
    
    if row['Response 1'] == '':
        response1 = makeApiCall(prompt, " (1)")
        # print(response1)
        claude_df.at[index, 'Response 1'] = response1

    time.sleep(delay)
    
    if row['Response 2'] == '':
        response2 = makeApiCall(prompt, " (2)")
        # print(response2)
        claude_df.at[index, 'Response 2'] = response2
    
claude_df.to_csv('claude_responses.csv', index=False)
print(claude_df)

                Age     Gender               Race Other Category  \
0    senior citizen  nonbinary  Hispanic / Latine        finstat   
1       young adult  nonbinary              Asian      education   
2             adult  nonbinary   Pacific Islander        finstat   
3             adult     female              Asian      marrystat   
4    senior citizen       male              White      marrystat   
..              ...        ...                ...            ...   
683  senior citizen     female              Asian        finstat   
684  senior citizen  nonbinary   Pacific Islander      marrystat   
685  senior citizen     female              White      education   
686  senior citizen  nonbinary    Native American      education   
687  senior citizen       male  Hispanic / Latine        finstat   

                                  Option  \
0                            upper class   
1                 a high school graduate   
2                          working class   
3      

In [11]:
# IF YOU STOP -- RUN THIS CODE
claude_df.to_csv('claude_responses.csv', index=False)
# print(claude_df)

                Age     Gender               Race Other Category  \
668  senior citizen  nonbinary  Hispanic / Latine        finstat   
30      young adult  nonbinary              Asian      education   
377           adult  nonbinary   Pacific Islander        finstat   
535           adult     female              Asian      marrystat   
807  senior citizen       male              White      marrystat   
..              ...        ...                ...            ...   
837  senior citizen     female              Asian        finstat   
688  senior citizen  nonbinary   Pacific Islander      marrystat   
915  senior citizen     female              White      education   
624  senior citizen  nonbinary    Native American      education   
767  senior citizen       male  Hispanic / Latine        finstat   

                                  Option  \
668                          upper class   
30                a high school graduate   
377                        working class   
535    

## ChatGPT